# ParallelLLMManager Comprehensive Demo

This notebook demonstrates the **ParallelLLMManager** - an advanced solution for parallel processing of AWS Bedrock requests across multiple models and regions. We'll showcase concurrent execution, load balancing, fault tolerance, and performance optimization.

## Key Features

- 🚀 **Parallel Processing**: Execute multiple requests concurrently across regions
- 🌍 **Multi-Region Support**: Intelligent load distribution across AWS regions  
- 🔄 **Load Balancing**: Round-robin, random, and least-loaded strategies
- 🛡️ **Fault Tolerance**: Multiple failure handling strategies
- 📊 **Performance Analytics**: Detailed execution statistics and metrics
- 🖼️ **Multimodal Support**: Parallel processing of text, image, and video content
- ⚙️ **Flexible Configuration**: Customizable concurrency and timeout settings
- 🔧 **MessageBuilder Integration**: Clean, fluent interface for creating requests

## Setup and Imports

In [ ]:
import sys
import json
import time
import asyncio
from pathlib import Path
import logging
from datetime import datetime
from typing import List, Dict, Any
import random

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import ParallelLLMManager and related classes
from bestehorn_llmmanager import ParallelLLMManager
from bestehorn_llmmanager.bedrock.models.parallel_structures import (
    BedrockConverseRequest, 
    ParallelProcessingConfig,
    FailureHandlingStrategy,
    LoadBalancingStrategy
)
from bestehorn_llmmanager.bedrock.models.llm_manager_structures import (
    AuthConfig, 
    RetryConfig, 
    AuthenticationType, 
    RetryStrategy
)
from bestehorn_llmmanager.bedrock.exceptions.parallel_exceptions import (
    ParallelProcessingError, 
    ParallelExecutionError, 
    ParallelConfigurationError
)

# Import MessageBuilder components
from bestehorn_llmmanager import create_user_message, create_assistant_message, create_message
from bestehorn_llmmanager import RolesEnum, ImageFormatEnum, DocumentFormatEnum, VideoFormatEnum

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")
print("🔧 MessageBuilder integration enabled for clean request creation!")

## Request Creation Functions

Using the MessageBuilder system to create requests cleanly and avoid serialization bugs.

In [ ]:
def create_text_request_with_builder(text: str, inference_config: Dict[str, Any] = None) -> BedrockConverseRequest:
    """Create a text-based BedrockConverseRequest using MessageBuilder."""
    message = create_user_message() \
        .add_text(text) \
        .build()
    
    return BedrockConverseRequest(
        messages=[message],
        inference_config=inference_config or {"maxTokens": 500, "temperature": 0.7}
    )

def create_image_request_with_builder(text: str, image_path: str, inference_config: Dict[str, Any] = None) -> BedrockConverseRequest:
    """Create an image analysis BedrockConverseRequest using MessageBuilder."""
    try:
        message = create_user_message() \
            .add_text(text) \
            .add_local_image(path_to_local_file=image_path, max_size_mb=5.0) \
            .build()
        
        return BedrockConverseRequest(
            messages=[message],
            inference_config=inference_config or {"maxTokens": 800, "temperature": 0.3}
        )
    except Exception as e:
        print(f"❌ Error creating image request: {e}")
        # Fall back to text-only request
        return create_text_request_with_builder(f"{text} (Note: Image could not be loaded from {image_path})")

def display_parallel_response(response, title="Parallel Response"):
    """Display a formatted parallel response."""
    print(f"\n{title}")
    print("=" * len(title))
    
    # Overall statistics
    print(f"✅ Overall Success: {response.success}")
    print(f"📊 Success Rate: {response.get_success_rate():.1f}%")
    print(f"⏱️  Total Duration: {response.total_duration_ms:.2f}ms")
    print(f"🔢 Total Requests: {len(response.request_responses)}")
    print(f"✅ Successful Requests: {len(response.get_successful_responses())}")
    print(f"❌ Failed Requests: {len(response.get_failed_responses())}")
    
    # Execution statistics
    if response.parallel_execution_stats:
        stats = response.parallel_execution_stats
        print(f"\n📈 Execution Statistics:")
        print(f"   ⚡ Avg Duration: {stats.average_request_duration_ms:.2f}ms")
        print(f"   📊 Max Duration: {stats.max_request_duration_ms:.2f}ms")
        print(f"   ⚡ Min Duration: {stats.min_request_duration_ms:.2f}ms")
        print(f"   🔀 Concurrent Executions: {stats.concurrent_executions}")
        
        if stats.region_distribution:
            print(f"   🌍 Region Distribution:")
            for region, count in stats.region_distribution.items():
                print(f"      {region}: {count} requests")
    
    # Token usage
    total_tokens = response.get_total_tokens_used()
    if any(total_tokens.values()):
        print(f"\n🎯 Total Token Usage:")
        for key, value in total_tokens.items():
            if value > 0:
                print(f"   {key}: {value:,}")
    
    # Average latency
    avg_latency = response.get_average_latency()
    if avg_latency:
        print(f"\n⚡ Average API Latency: {avg_latency:.2f}ms")
    
    # Warnings
    if response.warnings:
        print(f"\n⚠️  Warnings ({len(response.warnings)}):")
        for warning in response.warnings[:3]:  # Show first 3 warnings
            print(f"   - {warning}")
        if len(response.warnings) > 3:
            print(f"   ... and {len(response.warnings) - 3} more")

def display_individual_responses(response, max_responses=3):
    """Display individual responses from the parallel execution."""
    successful_responses = response.get_successful_responses()
    
    print(f"\n💬 Sample Individual Responses (showing up to {max_responses}):")
    print("-" * 50)
    
    count = 0
    for req_id, bedrock_response in successful_responses.items():
        if count >= max_responses:
            break
            
        print(f"\n🆔 Request ID: {req_id}")
        print(f"🤖 Model: {bedrock_response.model_used}")
        print(f"🌍 Region: {bedrock_response.region_used}")
        print(f"⏱️  Duration: {bedrock_response.total_duration_ms:.2f}ms")
        
        content = bedrock_response.get_content()
        if content:
            # Truncate long responses
            if len(content) > 200:
                content = content[:200] + "..."
            print(f"💭 Response: {content}")
        
        count += 1
    
    if len(successful_responses) > max_responses:
        print(f"\n... and {len(successful_responses) - max_responses} more successful responses")

print("✅ MessageBuilder-based helper functions defined!")

## Initialize the ParallelLLMManager

We'll create a ParallelLLMManager instance with multiple models and regions for optimal parallel processing.

In [ ]:
print("🚀 Initializing ParallelLLMManager...")

# Define models for parallel processing
models = [
    "Claude Haiku 4.5",
    "Claude Sonnet 4.5",
    "Nova Pro",             # Multimodal capabilities
    "Nova Lite"             # Backup option
]

# Define regions for load distribution
regions = [
    "us-east-1",
    "us-west-2", 
    "eu-west-1"
]

# Configure authentication
auth_config = AuthConfig(
    auth_type=AuthenticationType.PROFILE,
    profile_name="default"
)

# Configure retry behavior
retry_config = RetryConfig(
    max_retries=2,
    retry_strategy=RetryStrategy.REGION_FIRST
)

# Configure parallel processing behavior
parallel_config = ParallelProcessingConfig(
    max_concurrent_requests=8,
    request_timeout_seconds=60,
    failure_handling_strategy=FailureHandlingStrategy.CONTINUE_ON_FAILURE,
    load_balancing_strategy=LoadBalancingStrategy.ROUND_ROBIN,
    failure_threshold=0.3  # 30% failure threshold
)

try:
    # Initialize the ParallelLLMManager
    parallel_manager = ParallelLLMManager(
        models=models,
        regions=regions,
        auth_config=auth_config,
        retry_config=retry_config,
        parallel_config=parallel_config,
        timeout=30
    )
    
    print(f"✅ ParallelLLMManager initialized successfully!")
    print(f"   🤖 Models: {len(parallel_manager.get_available_models())}")
    print(f"   🌍 Regions: {len(parallel_manager.get_available_regions())}")
    print(f"   🔀 Max Concurrent: {parallel_config.max_concurrent_requests}")
    print(f"   ⚖️  Load Balancing: {parallel_config.load_balancing_strategy.value}")
    print(f"\n{parallel_manager}")
    
    # Validate configuration
    validation = parallel_manager.validate_configuration()
    print(f"\n🔍 Configuration Validation:")
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
    print(f"   Auth Status: {validation['auth_status']}")
    print(f"   Model-Region Combinations: {validation['model_region_combinations']}")
    print(f"   Parallel Config Valid: {'✅' if validation['parallel_config_valid'] else '❌'}")
    
    if validation['warnings'] or validation.get('parallel_warnings', []):
        all_warnings = validation['warnings'] + validation.get('parallel_warnings', [])
        print(f"   ⚠️  Warnings: {len(all_warnings)}")
        for warning in all_warnings[:2]:
            print(f"      - {warning}")
    
    if validation['errors'] or validation.get('parallel_errors', []):
        all_errors = validation['errors'] + validation.get('parallel_errors', [])
        print(f"   ❌ Errors: {len(all_errors)}")
        for error in all_errors:
            print(f"      - {error}")

except Exception as e:
    print(f"❌ Error initializing ParallelLLMManager: {e}")
    print("\n💡 Troubleshooting tips:")
    print("   1. Ensure AWS credentials are configured")
    print("   2. Check access to specified models and regions")
    print("   3. Verify network connectivity to AWS")
    raise

## Example 1: Basic Parallel Text Processing 💬

Let's start with parallel processing of multiple text requests using the MessageBuilder system.

In [ ]:
print("💬 Example 1: Basic Parallel Text Processing (MessageBuilder)")
print("=" * 58)

# Create multiple text requests using MessageBuilder
text_prompts = [
    "Explain quantum computing in simple terms.",
    "What are the benefits of renewable energy?",
    "Describe the process of photosynthesis.",
    "How does machine learning work?",
    "What is the significance of DNA in genetics?",
    "Explain the water cycle."
]

# Create BedrockConverseRequest objects using MessageBuilder
text_requests = []
for prompt in text_prompts:
    request = create_text_request_with_builder(text=prompt)
    text_requests.append(request)

print(f"📝 Created {len(text_requests)} text requests using MessageBuilder")
for i, req in enumerate(text_requests, 1):
    print(f"   {i}. {req.request_id}: {text_prompts[i-1][:50]}...")

try:
    # Execute requests in parallel
    print(f"\n🚀 Executing {len(text_requests)} requests in parallel...")
    start_time = time.time()
    
    parallel_response = parallel_manager.converse_parallel(
        requests=text_requests
        # Note: target_regions_per_request not specified - will auto-adjust based on 
        # available regions (3) and max_concurrent_requests (8), using min(8,3) = 3
    )
    
    end_time = time.time()
    sequential_estimate = len(text_requests) * 3  # Rough estimate: 3 seconds per request
    
    print(f"✅ Parallel execution completed in {end_time - start_time:.2f} seconds")
    print(f"⚡ Estimated sequential time: {sequential_estimate:.1f} seconds")
    print(f"🚀 Speed improvement: ~{sequential_estimate / (end_time - start_time):.1f}x faster")
    
    # Display results
    display_parallel_response(parallel_response, "🗣️  Parallel Text Processing Results")
    display_individual_responses(parallel_response, max_responses=2)

except Exception as e:
    print(f"❌ Error in parallel text processing: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 2: Parallel Image Analysis 🖼️

Now let's demonstrate parallel processing of image analysis requests using MessageBuilder's local image functionality.

In [ ]:
print("🖼️  Example 2: Parallel Image Analysis (MessageBuilder)")
print("=" * 50)

# Define image analysis tasks
image_tasks = [
    {
        "text": "Analyze the architectural features and historical significance of this landmark.",
        "image_path": "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"
    },
    {
        "text": "Describe the structural design and engineering aspects of this tower.",
        "image_path": "../images/Tokyo_Tower_2023.jpg"
    },
    {
        "text": "What can you tell me about the lighting and atmospheric conditions in this image?",
        "image_path": "../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg"
    },
    {
        "text": "Compare this tower's design to other famous towers around the world.",
        "image_path": "../images/Tokyo_Tower_2023.jpg"
    }
]

# Create image analysis requests using MessageBuilder
image_requests = []
for task in image_tasks:
    request = create_image_request_with_builder(
        text=task["text"],
        image_path=task["image_path"]
    )
    image_requests.append(request)

print(f"🖼️  Created {len(image_requests)} image analysis requests using MessageBuilder")
for i, task in enumerate(image_tasks, 1):
    image_name = Path(task["image_path"]).name
    print(f"   {i}. {image_name}: {task['text'][:40]}...")

try:
    # Execute image analysis in parallel
    print(f"\n🚀 Executing {len(image_requests)} image analysis requests in parallel...")
    start_time = time.time()
    
    image_response = parallel_manager.converse_parallel(
        requests=image_requests,
        target_regions_per_request=3  # Use more regions for vision models
    )
    
    end_time = time.time()
    
    print(f"✅ Parallel image analysis completed in {end_time - start_time:.2f} seconds")
    
    # Display results
    display_parallel_response(image_response, "🖼️  Parallel Image Analysis Results")
    display_individual_responses(image_response, max_responses=2)

except Exception as e:
    print(f"❌ Error in parallel image analysis: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 3: Performance Benchmarking 📊

Let's benchmark the performance improvements of parallel processing versus sequential execution using MessageBuilder.

In [ ]:
print("📊 Example 3: Performance Benchmarking (MessageBuilder)")
print("=" * 50)

# Create benchmark requests using MessageBuilder
benchmark_prompts = [
    "Summarize the history of computer science.",
    "Explain the principles of quantum mechanics.",
    "Describe the process of cellular respiration.",
    "What are the major causes of climate change?",
    "How do neural networks learn?",
    "Explain the theory of evolution.",
    "What is the importance of biodiversity?",
    "How does the internet work?"
]

benchmark_requests = []
for prompt in benchmark_prompts:
    request = create_text_request_with_builder(
        text=prompt,
        inference_config={"maxTokens": 400, "temperature": 0.6}
    )
    benchmark_requests.append(request)

print(f"📝 Created {len(benchmark_requests)} benchmark requests using MessageBuilder")

# Test 1: Sequential execution using single requests
print("\n🐌 Sequential Execution Test...")
sequential_start = time.time()
sequential_responses = []
sequential_success_count = 0

try:
    for i, request in enumerate(benchmark_requests[:4], 1):  # Limit to 4 for time
        print(f"   Processing request {i}/4...")
        response = parallel_manager.converse_with_request(request)
        sequential_responses.append(response)
        if response.success:
            sequential_success_count += 1
    
    sequential_end = time.time()
    sequential_duration = sequential_end - sequential_start
    
    print(f"✅ Sequential execution completed in {sequential_duration:.2f} seconds")
    print(f"   Success rate: {(sequential_success_count/4)*100:.1f}%")
    print(f"   Average per request: {sequential_duration/4:.2f} seconds")
    
except Exception as e:
    print(f"❌ Error in sequential execution: {e}")
    sequential_duration = float('inf')  # Set high value for comparison

# Test 2: Parallel execution
print("\n🚀 Parallel Execution Test...")
parallel_start = time.time()

try:
    parallel_response = parallel_manager.converse_parallel(
        requests=benchmark_requests[:4],  # Same 4 requests for fair comparison
        target_regions_per_request=2
    )
    
    parallel_end = time.time()
    parallel_duration = parallel_end - parallel_start
    
    print(f"✅ Parallel execution completed in {parallel_duration:.2f} seconds")
    print(f"   Success rate: {parallel_response.get_success_rate():.1f}%")
    
    # Performance comparison
    if sequential_duration != float('inf'):
        speedup = sequential_duration / parallel_duration
        efficiency = (speedup / parallel_config.max_concurrent_requests) * 100
        
        print(f"\n📈 Performance Comparison:")
        print(f"   Sequential: {sequential_duration:.2f}s")
        print(f"   Parallel: {parallel_duration:.2f}s")
        print(f"   Speedup: {speedup:.2f}x")
        print(f"   Parallel Efficiency: {efficiency:.1f}%")
        print(f"   Time Saved: {sequential_duration - parallel_duration:.2f}s ({((sequential_duration - parallel_duration)/sequential_duration)*100:.1f}%)")
    
    # Display detailed parallel statistics
    display_parallel_response(parallel_response, "📊 Benchmark Results")
    
except Exception as e:
    print(f"❌ Error in parallel execution: {e}")